In [17]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score

# Đọc tệp dữ liệu Cancer.csv trong file data_cancer
df = pd.read_csv('B:/Hanoi University of Science/Tài liệu ôn thi/2025-2026 Năm III kỳ 1/ML/Bài thực hành Tuần 2/bernoulli_nb_symptoms.csv')

#print(df.head() )


###### Tính các xác suất p_k := N_k / N (chú ý N = len(df) )

In [20]:
target_feature = 'condition'

Y = df[target_feature] 
labels = np.zeros((len(set(Y)), 2), dtype=Y.dtype) 

# Liệt kê các nhãn (label) khác nhau (từ trường cuisine) k =1, 2, ... C
# và tính xác suất P_k của mỗi nhãn. Sử dụng nhãn thay cho chỉ số
id = 0
for label in set(Y):
    labels[id, 0] = label
    labels[id, 1] = (Y == label).sum()/len(Y)
    id += 1

In [21]:
print(labels)

[['cold' np.float64(0.1576)]
 ['migraine' np.float64(0.0936)]
 ['allergy' np.float64(0.1184)]
 ['flu' np.float64(0.1532)]
 ['covid_like' np.float64(0.1532)]
 ['food_poison' np.float64(0.1064)]
 ['stomach_bug' np.float64(0.1236)]
 ['dehydration' np.float64(0.094)]]


###### Tính các giá trị P_ik ứng với xác suất để trường thứ i nhận giá trị 1: P_ik := [|{xm in class k : x_im = xi}| ] / [N_k]

In [13]:
def Pik_feature_per_class(X):
#Returns the prob. of feature per class
    X = np.array(X)
    # count the number of element x_m wich has field i^th : x_im = 1
    count = (X == 1).sum()
    # P_ik := [|{xm in class k : x_im = 1}|  / N_k 
    return count /len(X)
    

###### Lấy tên mỗi trường (field) của dữ liệu và số lựa chọn ứng với trường đó

In [6]:
def predict_output_label(X, x_input, p_labels, target_feature):

    #num_labels_M = get_distinct_value_in_fields()
    #score for each class 
    p = np.log(np.array(p_labels[:, 1], dtype=float))

    for k in range(len(p)):
        Xk = X[X[target_feature] == p_labels[k, 0]]
        
        for i in range(len(x_input)):
            Pik = Pik_feature_per_class(Xk.iloc[:, i+1])
            if(Pik != 0):
                p[k] += np.log(x_input.iloc[i] * Pik + (1 - x_input.iloc[i]) * (1 - Pik) )

    y_star = np.argmax(p)

    return p_labels[y_star, 0]

In [14]:
Train_size = round(len(df)*0.4)
X_Train = df[:Train_size]
Y_Train = df[target_feature][:Train_size]

X_Test = df[Train_size:].drop(target_feature, axis = 1).reset_index(drop=True, inplace=False)
Y_Test = df[target_feature][Train_size:].reset_index(drop=True, inplace=False)

Y_pred = Y_Test
#print(len(X_Test), len(Y_Test))
#print(X_Test.index, Y_Test.index)

for i in range(len(X_Test)):
    Y_pred[i] = predict_output_label(X_Train, X_Test.iloc[i, :], labels, target_feature)

print(Y_pred )

0              cold
1        covid_like
2          migraine
3               flu
4               flu
           ...     
1495        allergy
1496    food_poison
1497           cold
1498     covid_like
1499       migraine
Name: condition, Length: 1500, dtype: object


In [24]:
from sklearn.metrics import accuracy_score

print(accuracy_score(Y_pred, Y_Test))

1.0


In [16]:
print(predict_output_label(X_Train, X_Train.iloc[3, 1:], labels, target_feature) )

stomach_bug
